# K-Nearest Neighbors Regressor (KNN)

En este notebook entrenamos y evaluamos un **modelo de regresión basado en vecinos cercanos (KNN-Regressor)**.  
Este algoritmo predice el valor de una observación en función de la media de los valores de sus **k vecinos más cercanos**.

🔹 Características principales:
- Es un modelo **basado en distancias**, no asume relaciones lineales entre variables.
- Requiere que los datos estén **escalados**, porque las magnitudes afectan directamente al cálculo de distancias.
- El hiperparámetro clave es **k** (número de vecinos), que controla el sesgo y la varianza del modelo.


## **Paso 1. Importar librerías**

In [ ]:
# ===================================
# 1. Importar librerías
# ===================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns


## **Paso 2. Cargar datasets procesados (escalados) y divisón para entrenamiento**

Usamos el dataset escalado porque KNN calcula distancias entre observaciones y sería injusto que variables con magnitudes grandes dominaran el cálculo.

Si una variable tuviera valores mucho más grandes que las demás (ej. PIB vs. mortalidad), dominaría la distancia, sesgando el modelo.

Por eso cargamos el dataset **escalado** (`features_scaled.csv`), generado en el notebook `EDA_processed.ipynb` con `StandardScaler`.  

Esto asegura que todas las variables tengan media 0 y varianza 1, evitando que variables con unidades más grandes (por ejemplo PIB o gasto sanitario) dominen a otras más pequeñas.


**División en entrenamiento y validación**

En este bloque:

- Separamos el dataset en **conjunto de entrenamiento (80%)** y **conjunto de validación (20%)**.  
Esto nos permite entrenar el modelo con una parte de los datos y luego evaluar su rendimiento en datos que no ha visto antes.


De esta manera, podremos comparar los algoritmos de forma justa y evitar errores de entrenamiento.

- x = todas las columnas salvo la variable objetivo.
- y = la variable objetivo.
- train_test_split divide los datos en:
  - x_train, y_train (80%) → para entrenar el modelo.
  - x_valid, y_valid (20%) → para validar el modelo. 
  - random_state=42 → asegura que la división sea reproducible.

Este paso es obligatorio para poder comparar modelos en condiciones.

In [ ]:
# ===================================
# 2. Cargar datasets escalados
# ===================================
X = pd.read_csv("../data/processed/features_scaled.csv")
y = pd.read_csv("../data/processed/target_y.csv").squeeze()

# ===================================
# División en train/valid
# ===================================
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Tamaño entrenamiento:", X_train.shape, y_train.shape)
print("Tamaño validación:", X_valid.shape, y_valid.shape)


## **Paso 3. Entrenar y evaluar modelo (baseline con k=5)**

Entrenamos un modelo **KNN con k=5 vecinos** (valor por defecto habitual).  
Calculamos las métricas principales para evaluar su rendimiento:

- **RMSE**: raíz del error cuadrático medio.  
- **MAE**: error absoluto medio.  
- **R²**: porcentaje de varianza explicada por el modelo.  


In [ ]:
# ===================================
# 3. Entrenar y evaluar modelo baseline
# ===================================
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

preds = knn.predict(X_valid)

rmse = np.sqrt(mean_squared_error(y_valid, preds))
mae = mean_absolute_error(y_valid, preds)
r2 = r2_score(y_valid, preds)

print("📊 KNN Regressor (k=5)")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")
print(f"R²: {r2:.3f}")


## **Paso 4. Curva de validación para distintos valores de k**

Probamos diferentes valores de **k** (de 1 a 20) y evaluamos el error en validación.

Interpretación del gráfico:
- **k pequeño (1–3)**: el modelo es muy flexible pero se sobreajusta al ruido → error bajo en entrenamiento pero alto en validación.  
- **k grande (>15)**: el modelo es más estable pero pierde precisión → se vuelve demasiado "suavizado".  
- El mejor k se encuentra en el punto donde el **error de validación es mínimo**.  


In [ ]:
# ===================================
# 4. Curva de validación para distintos k
# ===================================
errors = []
neighbors = range(1, 21)

for k in neighbors:
    model = KNeighborsRegressor(n_neighbors=k)
    model.fit(X_train, y_train)
    preds_valid = model.predict(X_valid)
    rmse_k = np.sqrt(mean_squared_error(y_valid, preds_valid))
    errors.append(rmse_k)


plt.figure(figsize=(8,5))
plt.plot(neighbors, errors, marker="o")
plt.xlabel("Número de vecinos (k)")
plt.ylabel("RMSE en validación")
plt.title("Curva de validación KNN")
plt.show()


## **Paso 5. Entrenamiento final y evaluación de KNN para calcular métricas


**Entrenamiento con el mekor **k**

Después de explorar con el KNN Regressor cómo varía el error en función de k (número de vecinos), necesitamos entrenar el modelo con el mejor k, para predecir y calcular métricas (RMSE, MAE y R²).

- Seleccionamos el mejor valor de `k` a partir de la curva de validación (mínimo error en validación).  
- Entrenamos un modelo KNN con ese valor óptimo y evaluamos sus predicciones sobre el conjunto de validación.

📊 Las métricas que obtenemos son:
- **RMSE**: error cuadrático medio (raíz), penaliza más los errores grandes.  
- **MAE**: error absoluto medio, más robusto ante outliers.  
- **R²**: proporción de la variabilidad explicada por el modelo (idealmente cercano a 1).

📖 Interpretación de las métricas

Cuando entrenamos el KNN Regressor con el mejor k, obtenemos 3 métricas clave:

- **RMSE (Root Mean Squared Error)**:
      - Es la raíz del error cuadrático medio. Penaliza mucho los errores grandes.
      - Un RMSE bajo indica que las predicciones están cercanas a los valores reales.
    Si el RMSE es significativamente menor que el rango típico de la variable objetivo (vida media en años), es señal de buen ajuste.

- **MAE (Mean Absolute Error)**:
      - Es la media del error absoluto. Más robusto frente a outliers que el RMSE.
      - Indica cuántos "años" (de esperanza de vida) se equivoca, de media, el modelo.
    Si, por ejemplo, da 2.1, significa que el modelo suele errar en ±2 años, lo cual puede ser bastante aceptable.

- **R² (Coeficiente de determinación)**:
      - Mide la proporción de la varianza explicada por el modelo.
      - Valores cercanos a 1 implican que el modelo explica bien los datos.
    Un valor en torno a 0.9 o más sería muy bueno; valores bajos (<0.5) indicarían que el modelo no capta bien las relaciones.

👉 En resumen: si KNN devuelve métricas similares o mejores que regresión lineal y comparables a Random Forest/XGBoost, puede ser un candidato fuerte. Pero si empeora, será menos competitivo.

Esto nos permitirá comparar KNN con el resto de modelos en el notebook de comparativa final.


In [ ]:
# ===================================
# 5. Entrenamiento final y evaluación de KNN
# ===================================
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Escogemos el mejor k (mínimo error en validación)
best_k = errors.index(min(errors)) + 1
print(f"✅ Mejor número de vecinos (k): {best_k}")

# Entrenar modelo con ese k
knn_final = KNeighborsRegressor(n_neighbors=best_k)
knn_final.fit(X_train, y_train)

# Predicciones
preds_knn = knn_final.predict(X_valid)

# Métricas
rmse = np.sqrt(mean_squared_error(y_valid, preds_knn))
mae = mean_absolute_error(y_valid, preds_knn)
r2 = r2_score(y_valid, preds_knn)

print("\n📊 KNN Regressor (modelo final)")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")
print(f"R²: {r2:.3f}")


## **Paso 6. Gráfico de predicciones vs valores reales (KNN Regressor)**

Este gráfico es clave:

- Si los puntos se concentran **cerca de la línea roja discontinua (y = x)**, significa que las predicciones del modelo son bastante precisas y se acercan bien a los valores reales.  
- La **dispersión alrededor de la línea** indica errores en las predicciones: cuanto más lejos está un punto de la recta, mayor es la diferencia entre lo predicho y lo real.  
- Si aparece un **patrón sistemático** (por ejemplo, subestimar en valores altos o sobreestimar en valores bajos), refleja una limitación del modelo para generalizar en esas regiones.  
- Una **dispersión amplia y sin patrón claro** puede indicar que el modelo no está captando bien la complejidad de los datos y que sería necesario ajustar el hiperparámetro `k` (número de vecinos).  

👉 **En resumen**:  
- Una nube de puntos bien alineada con la recta implica **buen ajuste**.  
- Una dispersión notable sugiere **errores relevantes**.  
- Patrones de sub/sobreestimación muestran que el KNN podría necesitar **ajuste de parámetros** o que no es el algoritmo más adecuado para el dataset.  


In [ ]:
# ===================================
# 6. Visualización: predicciones vs valores reales
# ===================================
plt.figure(figsize=(6,6))
plt.scatter(y_valid, preds_knn, alpha=0.5)
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], "r--")
plt.xlabel("Valores reales (y_valid)")
plt.ylabel("Predicciones (KNN)")
plt.title("Predicciones vs Reales - KNN Regressor")
plt.show()


## **Paso 7. Optimización del hiperparámetro k en KNN Regressor con GridSearchCV

El parámetro más importante en **KNN Regressor** es `n_neighbors` (el número de vecinos considerados para predecir un valor).  
- Un `k` muy bajo (pocos vecinos) hace que el modelo sea muy sensible al ruido → **sobreajuste (overfitting)**.  
- Un `k` muy alto (muchos vecinos) suaviza demasiado las predicciones → **subajuste (underfitting)**.  

Usaremos **GridSearchCV** para probar distintos valores de `k` y encontrar el que minimiza el error.  


In [ ]:
# ===================================
# 7. GridSearchCV para optimizar KNN
# ===================================
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

# Definimos la rejilla de búsqueda
param_grid = {"n_neighbors": list(range(2, 31))}  # probamos k de 2 a 30

knn = KNeighborsRegressor()
grid_search = GridSearchCV(
    knn, 
    param_grid, 
    cv=5, 
    scoring="neg_root_mean_squared_error", 
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("📊 Mejor parámetro k encontrado:", grid_search.best_params_)
print(f"Mejor RMSE validación: {-grid_search.best_score_:.3f}")


## 📌 Interpretación de los resultados de GridSearchCV en KNN

El mejor valor encontrado para el parámetro `k` fue:

- **k = 3**
- **RMSE validación ≈ 2.958**

### 🔎 Lectura:
- Un valor tan bajo de `k` significa que el modelo está utilizando únicamente **3 vecinos más cercanos** para hacer cada predicción.  
- Esto indica que los datos **tienen suficiente estructura local** como para que pocos vecinos sean representativos.  
- Sin embargo, también es un indicio de que el modelo puede ser **sensible al ruido** (overfitting), ya que predice fuertemente influenciado por los vecinos más cercanos.  

### ✅ Conclusión:
- El rendimiento (RMSE ≈ 2.96) es **aceptable**, aunque no tan bueno como los modelos más potentes como **Random Forest o XGBoost**, que capturan relaciones más complejas.  
- Este resultado nos confirma que **KNN puede servir como baseline no lineal**, pero no es el modelo más robusto para este dataset.  

👉 Lo interesante será ver en el **notebook comparativo** cómo se posiciona KNN frente al resto de algoritmos.


## 📊 Curva de validación: RMSE vs. número de vecinos (k)

Este gráfico nos muestra cómo cambia el error (RMSE) en validación al variar el número de vecinos `k`.  

- Una curva en **forma de U** es habitual. Muestra cómo evoluciona el error (RMSE) en validación según el número de vecinos (`k`) del modelo **KNN Regressor**: 
  - Con **k muy bajo** (ej. 1-3), el modelo tiende a **sobreajustar** (overfitting).  
  - Con **k muy alto** (ej. 50+), el modelo tiende a **subajustar** (underfitting), ya que promedia demasiados vecinos.  
  - La **línea roja discontinua en k=3** marca el punto donde el RMSE es mínimo, indica el **k óptimo** según validación.
  - Este valor indica que, para nuestro dataset, el modelo alcanza su mejor rendimiento considerando **3 vecinos más cercanos**.  
  - A la izquierda de k=3, el modelo tiene tendencia a **sobreajustar**: se adapta demasiado al ruido de los datos y el error aumenta.  
  - A la derecha de k=3, el error empieza a crecer porque el modelo **subajusta**: al promediar demasiados vecinos, pierde la capacidad de capturar patrones locales.  

En conclusión, **k=3 ofrece el equilibrio óptimo entre sesgo y varianza** en este dataset, según la métrica RMSE.


In [ ]:
# ===================================
# 8. Curva de validación RMSE vs k
# ===================================
k_values = range(1, 31)
rmse_scores = []

for k in k_values:
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train, y_train)
    preds = knn.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, preds))
    rmse_scores.append(rmse)

plt.figure(figsize=(8,5))
plt.plot(k_values, rmse_scores, marker="o")
plt.axvline(3, color="red", linestyle="--", label="Mejor k=3")
plt.title("Curva de validación: RMSE vs k")
plt.xlabel("Número de vecinos (k)")
plt.ylabel("RMSE")
plt.legend()
plt.show()


## **Paso 9. Comparación de métricas según el número de vecinos (k)**

Hasta ahora hemos evaluado solo el **RMSE** en la curva de validación.  
Sin embargo, es útil observar también cómo evolucionan otras métricas como el **MAE** y el **R²** al variar `k`.

Esto nos permite:
- Confirmar si el valor óptimo de `k` es consistente en todas las métricas.  
- Entender mejor el comportamiento del modelo:  
  - **RMSE** penaliza más los errores grandes.  
  - **MAE** mide el error medio absoluto, más robusto a outliers.  
  - **R²** indica la proporción de varianza explicada por el modelo.  

El siguiente gráfico muestra la evolución de estas tres métricas a medida que aumentamos `k`.

Este gráfico nos dará una visión conjunta:
- Confirmará que k=3 es óptimo no solo en RMSE, sino también si se mantiene bajo en MAE y razonable en R².
- Si alguna métrica se comporta diferente, se deberá a una limitación del modelo.

In [ ]:
# ===================================
# Comparación de métricas con distintos k
# ===================================
from sklearn.metrics import r2_score

k_values = range(1, 21)
rmse_list, mae_list, r2_list = [], [], []

for k in k_values:
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train, y_train)
    preds = knn.predict(X_valid)
    
    rmse_list.append(np.sqrt(mean_squared_error(y_valid, preds)))
    mae_list.append(mean_absolute_error(y_valid, preds))
    r2_list.append(r2_score(y_valid, preds))

# Graficar resultados
plt.figure(figsize=(10,6))
plt.plot(k_values, rmse_list, marker="o", label="RMSE")
plt.plot(k_values, mae_list, marker="s", label="MAE")
plt.plot(k_values, r2_list, marker="^", label="R²")
plt.axvline(best_k, color="red", linestyle="--", label=f"Mejor k={best_k}")

plt.title("Evolución de métricas según k en KNN Regressor")
plt.xlabel("Número de vecinos (k)")
plt.ylabel("Valor de la métrica")
plt.legend()
plt.grid(True)
plt.show()




# ===================================
# Salvar métricas en CSV
# ===================================

# ===================================
# Función para guardar métricas estandarizadas
# ===================================
def save_results(model_name, y_true, y_pred, filepath):
    """
    Calcula métricas (RMSE, MAE, R²) y guarda en CSV estandarizado.
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    results = pd.DataFrame([{
        "Modelo": model_name,
        "RMSE": rmse,
        "MAE": mae,
        "R²": r2
    }])

    results.to_csv(filepath, index=False)
    print(f"✅ Resultados de {model_name} guardados en {filepath}")
    return results



# ===================================
# Guardar métricas finales de KNN (mejor k)
# ===================================


results_knn = save_results(
    "KNN (k=3)",
    y_valid,
    preds_knn,
    "../data/results_knn.csv"
)

print("📊 Resultados finales KNN (mejor k)")
print(f"Mejor k: {best_k}")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")
print(f"R²: {r2:.3f}")
print("✅ Resultados guardados en data/results_knn.csv")



## **Paso 10. Conclusiones**

- El rendimiento de KNN depende **fuertemente** de `k`.  
- Este modelo necesita datos **escalados**.  
- Funciona bien en datasets pequeños y de baja dimensionalidad, pero en grandes puede ser costoso porque calcula distancias con todos los puntos.  



👉 En resumen: si KNN devuelve métricas similares o mejores que regresión lineal y comparables a Random Forest/XGBoost, puede ser un candidato fuerte. Pero si empeora, será menos competitivo.


Compararemos este modelo con los demás (Lineal, Árboles, Random Forest, XGBoost, Ridge, Lasso, Elastic Net) en el notebook de **Comparación Final de Modelos**.